In [ ]:
import pandas as pd
import numpy as np
import string
import re
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
import janitor 
import geopandas as gpd

In [ ]:
# for csvs
#data_2020 = pd.read_csv("311_data/311_2020.csv")
#data_2021 = pd.read_csv("311_data/311_2021.csv")
#data_2022 = pd.read_csv("311_data/311_2022.csv")
#data_2023 = pd.read_csv("311_data/311_2023.csv")
#data_2024 = pd.read_csv("311_data/311_2024.csv")

In [ ]:
# for shapefiles
data_2020 = gpd.read_file("311_data/311_2020_shapes/311_2020.shp")
data_2021 = gpd.read_file("311_data/311_2021_shapes/311_2021.shp")
data_2022 = gpd.read_file("311_data/311_2022_shapes/311_2022.shp")
data_2023 = gpd.read_file("311_data/311_2023_shapes/311_2023.shp")
data_2024 = gpd.read_file("311_data/311_2024_shapes/311_2024.shp")


In [ ]:
datasets = [data_2020 , data_2021, data_2022, data_2023, data_2024]
for data in datasets:
    data = data.clean_names()

In [ ]:
# # Code copied from Ray's Feasability
# Create a dictionary to store all the data
data_dict = {
    2020: data_2020,
    2021: data_2021,
    2022: data_2022,
    2023: data_2023,
    2024: data_2024
}

for year, df in data_dict.items():
    print(f"data_{year}: {df.shape}")

# Check the consistency of the columns
for year, df in data_dict.items():
    print(f"{year}: {df.columns}")

### Standardize the column names

In [ ]:
# Rename some columns in data_2020
data_2020_clean = data_2020.drop(["service__1", "service__2", "FID", "public_url", "specify_th"], axis=1)
data_2020_clean.rename(columns={"X": "x",
                          "Y": "y",
                          "service_re": "service_request_id",
                          "sap_notifi": "sap_notification_number",
                          "case_age_d": "case_age_days",
                          "case_recor": "case_record_type",
                          "service_na": "service_name",
                          "street_add": "street_address",
                          "council_di": "council_district",
                          "comm_plan_": "comm_plan_code",
                          "comm_plan1": "comm_plan_name",
                          "case_origi": "case_origin",
                          "public_des": "public_description",
                          "date_reque": "date_requested",
                          "date_close": "date_closed",

                          }, inplace=True)





In [ ]:
# Only select the new 2020 columns from the other years
#print(data_2020_clean.columns)

data_2021_clean = data_2021.reindex(columns = data_2020_clean.columns)
#print(data_2021_clean.columns)
data_2022_clean = data_2022.reindex(columns = data_2020_clean.columns)
data_2023_clean = data_2023.reindex(columns = data_2020_clean.columns)
data_2024_clean = data_2024.reindex(columns = data_2020_clean.columns)

In [ ]:
# add a column for the year

Concatenate the 311 datasets by row (stacked on top of each other)

In [ ]:
data_concat = pd.concat([data_2020_clean, data_2021_clean, data_2022_clean, data_2023_clean, data_2024_clean])

In [ ]:
data_concat.head()

Create an variable on the proportion of letters that are all caps (besides the start of a sentence)

In [ ]:
def remove_sentence_starts(text):
    if pd.isna(text):
        return ""
    return re.sub(r'(^[A-Z])|(?<=[.!?]\s)[A-Z]', '', text)

no_sentence_starts = data_concat["public_description"].apply(remove_sentence_starts)
data_concat["prop_uppercase"] = (no_sentence_starts.str.count(r"[A-Z]"))/(no_sentence_starts.str.len())

In [ ]:
data_concat["prop_uppercase"]

In [ ]:
data_concat[["public_description","prop_uppercase"]]

In [ ]:
# From Ray's Feasability
def clean_text(df, col_name):
    return (
        df[col_name]
        .fillna("")     #Replaces all missing values with an empty string ""
        .str.lower()    #Converts all text to lowercase
        .str.replace(f"[{string.punctuation}]", "", regex=True) #Removes all punctuation marks from the text
        .str.replace(r"\s+", " ", regex=True) #Replaces multiple spaces or whitespace characters
        .str.strip()    #Removes leading and trailing spaces from each text string
    )

data_concat["clean_text"] = clean_text(data_concat, "public_description")
data_concat["word_count"] = data_concat["clean_text"].str.split().str.len()

In [ ]:
data_concat.head()
len(data_concat)

In [ ]:
# reproject to California Zone 6
data_concat = data_concat.to_crs(epsg=2230)

In [ ]:
# save to csv
#data_concat.to_csv("all_years_311.csv", index=False)

In [ ]:
# save to shp
data_concat.to_file("modified_data/all_years_311.shp")

### Preparing data to join with other variables

In [ ]:
#data_concat = gpd.read_file("modified_data/all_years_311.shp")

Spatial join with Land Use

In [ ]:
land_use = gpd.read_file("other_data/Land_Use_2024_shapefile/Land_Use_2024.shp")

In [ ]:
land_use.head()

In [ ]:
print(land_use.DESCRIPTIO.unique())
len(land_use.DESCRIPTIO.unique())

In [ ]:
land_use.plot(column="DESCRIPTIO")


Condense into fewer categories: 
- Residential
- Institutional/Group Quarters
- Industrial
- Commercial
- Transportation/Utilities
- Recreation/Open Space
- Agricultural
- Water/Natural Features
- Vacant/Under Construction

In [ ]:


land_use['land_use_binned'] = land_use['DESCRIPTIO'].case_when(
    [
        # Residential
       (land_use['DESCRIPTIO'].isin([
            'Spaced Rural Residential', 'Spaced Rural Residential Without Units',
            'Single Family Detached', 'Single Family Multiple-Units',
            'Single Family Residential Without Units', 'Multi-Family Residential',
            'Multi-Family Residential Without Units', 'Mobile Home Park',
            "Single Room Occupancy Units (SRO's)"
        ]), "Residential"),

        # Institutional / Group Quarters
        (land_use['DESCRIPTIO'].isin([
            'Jail/Prison', 'Dormitory', 'Military Barracks', 'Monastery',
            'Other Group Quarters Facility', 'UCSD/VA Hospital/Balboa Hospital',
            'Hospital - General', 'Other Health Care', 'Military Use',
            'Military Training', 'Weapons Facility', 'SDSU/CSU San Marcos/UCSD',
            'Other University or College', 'Junior College', 'Senior High School',
            'Junior High School or Middle School', 'Elementary School',
            'School District Office', 'Other School', 'Library', 'Post Office',
            'Mission', 'Religious Facility', 'Other Public Services', 'Fire/Police Station'
        ]), "Institutional"),

        # Industrial
        (land_use['DESCRIPTIO'].isin([
            'Heavy Industry', 'Industrial Park', 'Light Industry - General',
            'Warehousing', 'Public Storage', 'Extractive Industry',
            'Junkyard/Dump/Landfill'
        ]), "Industrial"),

        # Commercial
        (land_use['DESCRIPTIO'].isin([
            'Wholesale Trade', 'Regional Shopping Center', 'Community Shopping Center',
            'Neighborhood Shopping Center', 'Specialty Commercial',
            'Automobile Dealership', 'Arterial Commercial', 'Service Station',
            'Other Retail Trade and Strip Commercial', 'Office (High-Rise)',
            'Office (Low-Rise)', 'Government Office/Civic Center','Hotel/Motel (High-Rise)', 'Hotel/Motel (Low-Rise)', 'Resort',
            'Cemetery'
        ]), "Commercial"),

        # Mixed use
        (land_use['DESCRIPTIO'].isin(["Mixed Use"]), "Mixed Use"),

        # Transportation / Utilities
        (land_use['DESCRIPTIO'].isin([
            'Commercial Airport', 'Military Airport', 'General Aviation Airport',
            'Airstrip', 'Rail Station/Transit Center', 'Freeway', 'Road Right of Way',
            'Railroad Right of Way', 'Park and Ride Lot', 'Parking Lot - Surface',
            'Parking Lot - Structure', 'Other Transportation',
            'Communications and Utilities', 'Marine Terminal'
        ]), "Transportation"),

        # Recreation / Open Space
        (land_use['DESCRIPTIO'].isin([
            'Tourist Attraction', 'Stadium/Arena', 'Racetrack', 'Golf Course',
            'Golf Course Clubhouse', 'Convention Center', 'Marina',
            'Olympic Training Center', 'Casino', 'Other Recreation - High',
            'Other Recreation - Low', 'Park - Active', 'Open Space Park or Preserve',
            'Beach - Active', 'Beach - Passive', 'Landscape Open Space',
            'Residential Recreation', 'Undevelopable Natural Area'
        ]), "Rec / Open Space"),

        #  Agricultural
        (land_use['DESCRIPTIO'].isin([
            'Orchard or Vineyard', 'Intensive Agriculture', 'Field Crops'
        ]), "Agricultural"),

        #  Water / Natural
        (land_use['DESCRIPTIO'].isin([
            'Water', 'Bay or Lagoon', 'Lake/Reservoir/Large Pond'
        ]), "Water"),


        #  Vacant 
        (land_use['DESCRIPTIO'].isin([
            'Vacant and Undeveloped Land','Residential Under Construction', 'School Under Construction',
            'Industrial Under Construction', 'Office Under Construction', 'Commercial Under Construction',
            'Freeway Under Construction', 'Road Under Construction'
        ]), "Vacant / Under Construction")
    ]
    
)


In [ ]:
land_use.head()

In [ ]:
land_use.crs

Join with 311 data

In [ ]:
land_use_311 = gpd.sjoin(data_concat, land_use, how="left", predicate="within")

In [ ]:
land_use_311.head()

In [ ]:
# save to shp
land_use_311.to_file("modified_data/augmented_311.shp")

### Add binary variable: whether in park or not 

In [ ]:
augmented_311 = gpd.read_file("modified_data/augmented_311.shp")


In [ ]:
parks = gpd.read_file("other_data/Parks_SD_shapefile/Parks_SD.shp")

In [ ]:
# match projection
parks = parks.to_crs(augmented_311.crs)

In [ ]:
# spatial join
parks_311 = gpd.sjoin(augmented_311, parks, how="left", predicate = "within")





In [ ]:
parks_311['in_park'] = parks_311['index_right'].notna().astype(int)

In [ ]:
# create binary column: if a report is in a park, it will have an index_right value
selected_cols = list(augmented_311.columns) + ['in_park']
result = parks_311[selected_cols]

In [ ]:
result.head()

In [ ]:
result.to_file("modified_data/augmented_311.shp")